In [1]:
import csv
import pandas as pd
import clip
import torch
import redis
import numpy as np
import pickle
import sys


In [2]:
dataset = 'things-eeg-small'
subject = 'sub-08'
path = f'/root/workspace/wht/multimodal_brain/datasets/things-eeg-small/Preprocessed_data_250Hz/{subject}/train.pt'
data = torch.load(path)
data.keys()

dict_keys(['eeg', 'session', 'img', 'text', 'repeat_id', 'class_label', 'ch_names', 'times'])

In [3]:
for key in ['eeg','session','img','text','repeat_id','class_label']:
    print(data[key].dtype)

float64
int64
<U74
<U18
int64
int64


In [4]:
r = redis.Redis(host='localhost', port=6379, db=0)

In [31]:
for i in range(len(data['eeg'])):
    _data = {
        "eeg": pickle.dumps(data['eeg'][i].astype(np.float32)),          
        "session":int(data['session'][i]),         
        "img": data['img'][i],         
        "text": data['text'][i],         
        "repeat_id": int(data['repeat_id'][i]),         
        "class_label": int(data['class_label'][i]),         
    }
    r.hset(f"eeg_{dataset}_{subject}_{i}", mapping=_data)

In [30]:
r.flushall()


True

In [40]:
import time,random
db_size = r.dbsize()
print("Number of keys in the database:", db_size)
# print(r.keys())

start_time = time.time()
keys = r.keys()

random.shuffle(keys)

for key in r.keys():
    key = key.decode('utf-8')
    value = r.hgetall(key)
    # for k,v in value.items():
    #     k =k.decode('utf-8')
    #     if k == 'eeg':
    #         v = pickle.loads(v)
    #         pass
    #     else:
    #         v =v.decode('utf-8')
    #     if k in ['session','repeat_id','class_label']:
    #         v = int(v)
end_time = time.time()
print(f"时间: {end_time - start_time} 秒")

Number of keys in the database: 66160
时间: 76.6370165348053 秒


In [ ]:
cursor, keys = r.scan(cursor=0)
print(len(keys))

In [ ]:
import numpy as np
import pickle
import time


# 重复次数
repeats = 60000

# 序列化和反序列化时间列表
serialization_times = []
deserialization_times = []

for _ in range(repeats):
    # 创建一个随机的63x250的NumPy矩阵
    matrix = np.random.rand(63, 250)

    # 序列化时间
    start_time = time.time()
    serialized_matrix = pickle.dumps(matrix)
    serialization_time = time.time() - start_time
    serialization_times.append(serialization_time)
    
    # 反序列化时间
    start_time = time.time()
    deserialized_matrix = pickle.loads(serialized_matrix)
    deserialization_time = time.time() - start_time
    deserialization_times.append(deserialization_time)

# 计算平均时间
avg_serialization_time = sum(serialization_times) 
avg_deserialization_time = sum(deserialization_times) 

print("Average Serialization Time:", avg_serialization_time, "seconds")
print("Average Deserialization Time:", avg_deserialization_time, "seconds")